# Výslovnostní varianty v angličtině

Jeden z lexikálních korpusů (= slovníků) dostupných v NLTK je anglický výslovnostní slovník *CMU Pronouncing Dictionary*:

In [1]:
from nltk.corpus import cmudict

Podobně jako k ostatním korpusům k němu lze přistupovat např. po jednotlivých slovech:

In [2]:
words = cmudict.words()
words[:10]

['a',
 'a.',
 'a',
 'a42128',
 'aaa',
 'aaberg',
 'aachen',
 'aachener',
 'aaker',
 'aalseth']

Užitečnější ale je nechat si data načíst strukturovaná tak, abychom si mohli prohlížet jednotlivé záznamy ve slovníku. K tomu slouží metoda `.dict()`:

In [3]:
vyslovnosti = cmudict.dict()

Nadefinujme si funkci, která nám umožní nahlédnout do obsahu pythonovského slovníku (bez toho, aby se nám zbytečně vytiskl celý a zabíral hromadu místa v notebooku).

In [4]:
def dict_peek(d, max_items=5):
    peek = {}
    for i, (k, v) in enumerate(d.items()):
        if i == max_items:
            break
        peek[k] = v
    return peek    

In [5]:
dict_peek(vyslovnosti)

{'carpetbaggers': [['K',
   'AA1',
   'R',
   'P',
   'AH0',
   'T',
   'B',
   'AE2',
   'G',
   'ER0',
   'Z']],
 'crest': [['K', 'R', 'EH1', 'S', 'T']],
 'depressurizes': [['D',
   'IH0',
   'P',
   'R',
   'EH1',
   'SH',
   'ER0',
   'AY2',
   'Z',
   'IH0',
   'Z']],
 'frain': [['F', 'R', 'EY1', 'N']],
 'marquette': [['M', 'AA0', 'R', 'K', 'EH1', 'T']]}

**POZN.:** U seznamů taková funkce není třeba, pro náhled stačí vyříznout část seznamu pomocí syntaxe `seznam[:5]`.

Díky slovníku `vyslovnosti` pak můžeme zjistit výslovnost cílového slova...

In [6]:
vyslovnosti["cat"]

[['K', 'AE1', 'T']]

... a případně i výslovnostní varianty, pokud je kanonických výslovností zachycených ve slovníku víc:

In [7]:
vyslovnosti["fire"]

[['F', 'AY1', 'ER0'], ['F', 'AY1', 'R']]

Jaká proporce slov ve slovníku CMU má dvě a více výslovností? Které/á slovo/a mají nejvíce výslovnostních variant?

In [8]:
# jen proporce
multi = 0

for varianty in vyslovnosti.values():
    if len(varianty) > 1:
        multi += 1

multi / len(vyslovnosti)

0.07485318537118789

In [9]:
# proporce včetně seznamu slov, která mají více výslovností
multi = []

for varianty in vyslovnosti.values():
    if len(varianty) > 1:
        multi.append(varianty)

len(multi) / len(vyslovnosti)

0.07485318537118789

In [10]:
multi[:5]

[[['HH', 'AE1', 'L'], ['HH', 'AE1', 'L', 'IY0']],
 [['EH2', 'JH', 'AH0', 'K', 'EY1', 'SH', 'AH0', 'N', 'Z'],
  ['EH2', 'JH', 'Y', 'UW0', 'K', 'EY1', 'SH', 'AH0', 'N', 'Z']],
 [['P', 'ER0', 'AA1', 'T', 'IH0', 'S'], ['P', 'ER0', 'AE1', 'T', 'IH0', 'S']],
 [['B', 'R', 'EH1', 'N', 'T', 'AH0', 'L', 'IH0', 'NG', 'ER0'],
  ['B', 'R', 'EH1', 'N', 'T', 'L', 'IH0', 'NG', 'ER0']],
 [['TH', 'ER1', 'Z', 'D', 'IY0', 'Z'], ['TH', 'ER1', 'Z', 'D', 'EY2', 'Z']]]

In [11]:
# varianta, kdy ve for-cyklu místo hodnot (values), tj. výslovností,
# procházíme (a do seznamu ukládáme) klíče (keys), tj. ortograficky
# zapsaná slova
multi = []

# klíče jsou při procházení slovníku přes `for ... in` default, ale
# možno též si explicitně vyžádat vyslovnosti.keys()
for word in vyslovnosti:
    if len(vyslovnosti[word]) > 1:
        multi.append(word)

len(multi) / len(vyslovnosti)

0.07485318537118789

---

## Odbočka

Kromě ostré nerovnosti (`<`, `>`) umí Python pochopitelně i nerovnost neostrou (`<=`, `>=`):

In [12]:
1 <= 1

True

Případně lze dokonce velmi úsporně zapsat i test, zda číslo spadá do nějakého intervalu:

In [13]:
číslo = 5
1 < číslo < 3

False

---

Pokud chceme roztřídit výslovnosti podle počtu variant, můžeme si předpřipravit seznamy pro různé počty variant a výslovnosti do nich rozdělit:

In [14]:
multi = []
dvě = []
tři = []

for varianty in vyslovnosti.values():
    length = len(varianty)
    if length == 2:
        dvě.append(varianty)
    elif length == 3:
        tři.append(varianty)
    # POZOR, tady chyba viz níže
    elif length > 1:
        multi.append(varianty)

# kvůli chybě zde jiný výsledek než 0.074...
len(multi) / len(vyslovnosti)

0.0017334251346644526

Sled `if / elif / else` představuje jedno rozvětvení, z něhož se vykoná vždy jen jedno rameno (první v pořadí, jehož podmínka se ukáže být pravdivá):

```
     .---> if
    /
   /-----> elif
  /
 --------> elif
  \
   \-----{ ...
    \
     `---> else
```

V kódu výše tedy do seznamu `multi` přidáme pouze ty `varianty`, které nespadly už do dvou předchozích ramen rozvětvení. Tzn. v kódu sice testujeme `length > 1`, ale reálně sem spadnou jen `varianty` s `length` 4 a více, protože ty s `length` 2 a 3 už spadly do dvou předchozích ramen.

Řešení je jednoduché: použít místo `elif` nové holé `if`, čím započneme nové rozvětvení, které je nezávislé na tom předchozím:

```
    .---> if
   /
  /-----> elif
 /
--------> elif
 \
  \-----{ ...
   \
    `---> else

--------> if
```

In [15]:
multi = []
dvě = []
tři = []

for varianty in vyslovnosti.values():
    length = len(varianty)
    # první rozvětvení, sloužící k roztřídění do kategorií podle
    # počtu variant
    if length == 2:
        dvě.append(varianty)
    elif length == 3:
        tři.append(varianty)
    # druhé rozvětvení, sloužící k odchycení všech výslovností s více
    # než jednou variantou
    if length > 1:
        multi.append(varianty)

# teď je výsledek zase správně (0.74...)
len(multi) / len(vyslovnosti)

0.07485318537118789

Přístup s předdefinovanými seznamy má ale nevýhodu v tom, že musím dopředu vědět (nebo si tipnout), jaké s jakými počty variant se v datech setkáme (2, 3, 4, 5...?), všem jim předem nadefinovat seznamy a pak rozdělení ručně zrcadlit v rozepsané sekvenci `if / elif / elif ...`, což snadno povede k chybě (překlep, nedoupravené `Ctrl + C, Ctrl + V` apod.).

Řešením je vytvořit si slovník, v němž si budeme **dynamicky** (tj. podle hodnot skutečně nalezených v datech) vytvářet a doplňovat seznamy pro jednotlivé počty variant. Klíči v onom slovníku budou právě ony počty:

In [16]:
podle_delky = {}

for varianty in vyslovnosti.values():
    length = len(varianty)
    if length > 1:
        # pokud jsme tuto délku ještě neviděli...
        if length not in podle_delky:
            # ... tak ji do slovníku přidáme jako klíč a namapujeme
            # na ni nový prázdný seznam
            podle_delky[length] = []
        # vytáhneme ze slovníku seznam odpovídající aktuálně
        # naměřené délce...
        seznam = podle_delky[length]
        # ... a přidáme do něj aktuální varianty
        seznam.append(varianty)
        # předchozí dva řádky jdou ve zkratce zapsat také:
        # podle_delky[length].append(varianty)

# slovník má pak jako klíče jednotlivé nalezené počty variant...
podle_delky.keys()

dict_keys([2, 3, 4, 5])

In [17]:
# ... a jako hodnoty výslovnosti s odpovídajícím počtem variant
podle_delky[5]

[[['F', 'EH1', 'B', 'Y', 'AH0', 'W', 'EH2', 'R', 'IY0', 'Z'],
  ['F', 'EH1', 'B', 'AH0', 'W', 'EH2', 'R', 'IY0', 'Z'],
  ['F', 'EH1', 'B', 'R', 'UW0', 'W', 'EH2', 'R', 'IY0', 'Z'],
  ['F', 'EH1', 'B', 'UW0', 'W', 'EH2', 'R', 'IY0', 'Z'],
  ['F', 'EH1', 'B', 'Y', 'UW0', 'W', 'EH2', 'R', 'IY0', 'Z']],
 [['F', 'EH1', 'B', 'Y', 'AH0', 'W', 'EH2', 'R', 'IY0'],
  ['F', 'EH1', 'B', 'AH0', 'W', 'EH2', 'R', 'IY0'],
  ['F', 'EH1', 'B', 'R', 'UW0', 'W', 'EH2', 'R', 'IY0'],
  ['F', 'EH1', 'B', 'UW0', 'W', 'EH2', 'R', 'IY0'],
  ['F', 'EH1', 'B', 'Y', 'UW0', 'W', 'EH2', 'R', 'IY0']]]

In [18]:
n = 5
print("In the CMU dictionary, there is/are", len(podle_delky[n]), "pronunciation(s) with", n, "variants.")

In the CMU dictionary, there is/are 2 pronunciation(s) with 5 variants.


Protože sekvence operací na řádcích 7--13 v buňce 16 je poměrně častá, mají slovníky speciální metodu `.setdefault()`, která ve zkrácené podobě udělá totéž. Jinými slovy, kód `hodnota = slovnik.setdefault(x, y)` vykoná následující operace:

> - podívej se, zda ve slovníku existuje klíč `x`
> - pokud tam není, tak do slovníku ke klíči `x` přidej nějakou defaultní hodnotu `y`
> - a pak mi na závěr vrať hodnotu, která ve slovníku klíči `x` odpovídá, ať už je to právě vložené `y` nebo nějaké `z`, které tam bylo už dříve

In [21]:
slovnik = {}
hodnota = slovnik.setdefault("x", "y")
hodnota

'y'

In [22]:
slovnik = {"x": "z"}
hodnota = slovnik.setdefault("x", "y")
hodnota

'z'

In [24]:
# v našem případě tedy:
podle_delky = {}

for varianty in vyslovnosti.values():
    length = len(varianty)
    if length > 1:
        # místo řádků 7--13 v buňce 16 stačí jen následující rádek
        seznam = podle_delky.setdefault(length, [])
        seznam.append(varianty)
        # předchozí dva řádky můžeme znovu zkondenzovat, a to následovně:
        # podle_delky.setdefault(length, []).append(varianty)

podle_delky.keys()

dict_keys([2, 3, 4, 5])

In [25]:
podle_delky[5]

[[['F', 'EH1', 'B', 'Y', 'AH0', 'W', 'EH2', 'R', 'IY0', 'Z'],
  ['F', 'EH1', 'B', 'AH0', 'W', 'EH2', 'R', 'IY0', 'Z'],
  ['F', 'EH1', 'B', 'R', 'UW0', 'W', 'EH2', 'R', 'IY0', 'Z'],
  ['F', 'EH1', 'B', 'UW0', 'W', 'EH2', 'R', 'IY0', 'Z'],
  ['F', 'EH1', 'B', 'Y', 'UW0', 'W', 'EH2', 'R', 'IY0', 'Z']],
 [['F', 'EH1', 'B', 'Y', 'AH0', 'W', 'EH2', 'R', 'IY0'],
  ['F', 'EH1', 'B', 'AH0', 'W', 'EH2', 'R', 'IY0'],
  ['F', 'EH1', 'B', 'R', 'UW0', 'W', 'EH2', 'R', 'IY0'],
  ['F', 'EH1', 'B', 'UW0', 'W', 'EH2', 'R', 'IY0'],
  ['F', 'EH1', 'B', 'Y', 'UW0', 'W', 'EH2', 'R', 'IY0']]]

In [26]:
# a na závěr, pokud si chceme kromě výslovnostních variant uložit
# i ortografický zápis...
podle_delky = {}

for word, varianty in vyslovnosti.items():
    length = len(varianty)
    if length > 1:
        seznam = podle_delky.setdefault(length, [])
        # ... stačí si do seznamů místo pouhých variant ukládat
        # dvojice (ortografický_zápis, seznam_variant)
        seznam.append((word, varianty))

podle_delky[5]

[("february's",
  [['F', 'EH1', 'B', 'Y', 'AH0', 'W', 'EH2', 'R', 'IY0', 'Z'],
   ['F', 'EH1', 'B', 'AH0', 'W', 'EH2', 'R', 'IY0', 'Z'],
   ['F', 'EH1', 'B', 'R', 'UW0', 'W', 'EH2', 'R', 'IY0', 'Z'],
   ['F', 'EH1', 'B', 'UW0', 'W', 'EH2', 'R', 'IY0', 'Z'],
   ['F', 'EH1', 'B', 'Y', 'UW0', 'W', 'EH2', 'R', 'IY0', 'Z']]),
 ('february',
  [['F', 'EH1', 'B', 'Y', 'AH0', 'W', 'EH2', 'R', 'IY0'],
   ['F', 'EH1', 'B', 'AH0', 'W', 'EH2', 'R', 'IY0'],
   ['F', 'EH1', 'B', 'R', 'UW0', 'W', 'EH2', 'R', 'IY0'],
   ['F', 'EH1', 'B', 'UW0', 'W', 'EH2', 'R', 'IY0'],
   ['F', 'EH1', 'B', 'Y', 'UW0', 'W', 'EH2', 'R', 'IY0']])]